In [10]:
import spacy
from spacy.matcher import Matcher
import json
from os.path import join
from tqdm.auto import tqdm
from QuickUMLS.quickumls import QuickUMLS

from utils import (
    extract_UMLS_types,
    get_data_dir,
    load_if_not_present,
    QUICKUMLS_LOCATION_WIDGET, 
    save_df, 
    DATALOCATION_WIDGET, 
    SESSIONLOCATION_WIDGET
)


from abstract_manipulations import (
    unicode2ascii, 
    load_unicode_mappings, 
    get_full_text, 
    replace_abstract_abbreviations,
    get_relevant_text
)

from filter_sentences import (
    get_abstract_generator,
    get_relevant_sentences_generator,
    get_sentence_generator,
    get_relevant_sentences_generator
)

from abstracts_spacy import set_extensions,ner_on_abstract

tqdm.pandas()



In [2]:
print("Enter the directory where data can be found.")
display(DATALOCATION_WIDGET)
print("Enter subdirectory for this specific session. If nonexistant it will be created.")
display(SESSIONLOCATION_WIDGET)

Enter the directory where data can be found.


Text(value='data', description='Data location:')

Enter subdirectory for this specific session. If nonexistant it will be created.


Text(value='small_subset', description='Session location:')

In [3]:
nlp = spacy.load("en_core_sci_md")

In [4]:
if not spacy.tokens.Doc.has_extension("contains_relation_indicator"):
    spacy.tokens.Doc.set_extension("contains_relation_indicator",default=False)

if not spacy.tokens.Doc.has_extension("ignore"):
    spacy.tokens.Doc.set_extension("ignore",default=False)

In [5]:
abstracts_noparens = None
abstracts_noparens = load_if_not_present(
    abstracts_noparens, "abstracts_noparens.csv")

Loaded abstracts_noparens.csv from csv.


In [6]:
abstract_generator = get_abstract_generator(abstracts_noparens)

In [7]:
abstract_doc_generator = nlp.pipe(abstract_generator,batch_size=32,disable=["ner"],as_tuples=True)

In [8]:
sent_generator = get_sentence_generator(abstract_doc_generator)

In [9]:
rel_sent_generator = get_relevant_sentences_generator(sent_generator,nlp)

Make a file with interesting sentences to train entity recognition:

In [15]:
with open("data/sessions/small_subset/relevant_sentences_and_pmid.jsonl","w+") as f:
    for sentence,pmid in tqdm(rel_sent_generator):
        obj = {
            "sentence": sentence.text,
            "pmid": pmid
        }
        js = json.dumps(obj)
        f.write(js)
        f.write("\n")
